<a href="https://colab.research.google.com/github/hereagain-Y/TF_tutorial/blob/main/VAE_tutorial(keras).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np




In [ ]:
data=pd.read_csv("/content/drive/My Drive/DL/VAE/rep_00148.tsv",
            sep="\t")
d= {} # bipphysics dictionary
with open("/content/drive/My Drive/DL/VAE/AAidx_PCA.txt") as f:
    next(f)
    for line in f.readlines():
        line=line.strip().split('\t') #
        AA=line[0]
        tag=0
        values=[]
        for PC in line[1:]:
            values.append(float(PC))
        if tag==1: 
            continue
        d[AA]=values
d # AA 

In [ ]:
# max length
comp_seq=data["amino_acid"].tolist()
max_len=-1 

for AA in comp_seq:
    if len(AA)>max_len:
        max_len=len(AA)
        #res=AA
print(max_len )


19


In [ ]:
PC_length=len(d['C'])
def AAindexEncoding(Seq):
    length_seq=len(Seq)
    global max_len
    AAE=np.zeros([max_len,15])
    if length_seq<max_len:
        for amino in range(length_seq):
            AA=Seq[amino]# 
            AAE[amino,]=d[AA] # add PC value 
            
        for amino in range(length_seq,max_len):
            AAE[amino,]=np.zeros(15)
    else: 
        for amino in range(length_seq):
            AA=Seq[amino]# 
            AAE[amino,]=d[AA]
        
    #AAE=np.transpose(AAE.astype(np.float32)) # row as PC. and column as AA sequence 
    return AAE 
  
def GetFeatures(file):
    sequence=file['amino_acid'].tolist()
    sequence=np.array(sequence)
    hot_encode=[]
    for seq in sequence:
        hot_encode.append(AAindexEncoding(seq))
    hot_encode=np.array(hot_encode)
    result=np.array(hot_encode)
    return(result) # dimension: number of sequence [15*length(sequence)]


r1=GetFeatures(data)
r1

array([[[-1.62976860e+01, -3.90769937e+00, -1.37925549e+01, ...,
         -1.94456110e+00, -8.14384633e-01,  1.37250825e+00],
        [ 1.62170472e+01,  1.50912704e+01, -8.72320617e+00, ...,
         -4.38115415e-01,  5.20347513e-02,  5.05554282e-01],
        [ 1.62170472e+01,  1.50912704e+01, -8.72320617e+00, ...,
         -4.38115415e-01,  5.20347513e-02,  5.05554282e-01],
        ...,
        [ 7.92109650e+00, -8.69666506e+00, -5.97016945e-01, ...,
          2.44305793e+00, -2.76002278e+00, -1.69871720e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00]],

       [[ 1.62170472e+01,  1.50912704e+01, -8.72320617e+00, ...,
         -4.38115415e-01,  5.20347513e-02,  5.05554282e-01],
        [-1.76362377e+01, -3.52395967e-01,  1.18392432e+01, ...,
          1.45479040e+00,  1.28288187e

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Layer, Conv2D, Flatten, Dense, Reshape, Conv2DTranspose
from tensorflow.keras import layers

In [ ]:
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

In [ ]:
latent_dim = 32

encoder_inputs = keras.Input(shape=(15, 18, 1))
x = layers.Flatten()(encoder_inputs)
x = layers.Dense(128, activation=None)(x)
x= layers.Dense(64, activation="relu")(x)
z_mean = layers.Dense(32, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
z = Sampling()([z_mean, z_log_var])
encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
encoder.summary()

Model: "encoder"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 15, 18, 1)]  0           []                               
                                                                                                  
 flatten (Flatten)              (None, 270)          0           ['input_1[0][0]']                
                                                                                                  
 dense (Dense)                  (None, 128)          34688       ['flatten[0][0]']                
                                                                                                  
 dense_1 (Dense)                (None, 64)           8256        ['dense[0][0]']                  
                                                                                            

In [ ]:

latent_inputs = keras.Input(shape=(latent_dim,))
x = layers.Dense(64, activation="relu")(latent_inputs)
#x = layers.Reshape((7, 7, 64))(x)
x = layers.Dense(128, activation=None)(x)
x=layers.Dense(270, activation=None)(x)
decoder_outputs = layers.Reshape((15,18,1))(x)
decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")
decoder.summary()

Model: "decoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 32)]              0         
                                                                 
 dense_2 (Dense)             (None, 64)                2112      
                                                                 
 dense_3 (Dense)             (None, 128)               8320      
                                                                 
 dense_4 (Dense)             (None, 270)               34830     
                                                                 
 reshape (Reshape)           (None, 15, 18, 1)         0         
                                                                 
Total params: 45,262
Trainable params: 45,262
Non-trainable params: 0
_________________________________________________________________


In [ ]:
class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            reconstruction_loss = tf.reduce_mean(
                tf.reduce_sum(
                    keras.losses.binary_crossentropy(data, reconstruction), axis=(1, 2)
                )
            )
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

In [ ]:
# train the model 
#x_train, x_test=
#vae=VAE(encoder,decoder)
#vae.compile(optimizer=keras.optimizers.Adam())
#vae.fit(data, epochs=30, batch_size=128)